In [1]:
import os
from glob import glob
import dask
import xarray as xr
import numpy as np
from netCDF4 import Dataset
from scipy.io import loadmat

from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)

w = cluster.start_workers(1)

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/utils.py:128: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  % (host, default, e), RuntimeWarning)


In [3]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [4]:
client

Client Scheduler: tcp://10.148.1.37:45699 Dashboard: http://10.148.1.37:8787/status,Cluster Workers: 2 Cores: 8 Memory: 100.00 GB


# Grid

In [5]:
# the path where (Zach's) LLC4320 data are already stored
LLC_data_folder = '/home1/datawork/xyu/OSMOSIS_llc4320/'
# the path where the grid data will be re-stored
LLC_data_folder_data_store = '/home1/datawork/xyu/OSMOSIS_llc4320/data_store/'

In [6]:
XC = xr.DataArray(np.fromfile(LLC_data_folder+"grid/XC_177x201", dtype='>f').reshape(201,177), dims=['lat','lon'])
XC = np.real(XC).rename('XC')

YC = xr.DataArray(np.fromfile(LLC_data_folder+"grid/YC_177x201", dtype='>f').reshape(201,177), dims=['lat','lon'])
YC = np.real(YC).rename('YC')

XG = xr.DataArray(np.fromfile(LLC_data_folder+"grid/XG_177x201", dtype='>f').reshape(201,177), dims=['lat_g','lon_g'])
XG = np.real(XG).rename('XG')

YG = xr.DataArray(np.fromfile(LLC_data_folder+"grid/YG_177x201", dtype='>f').reshape(201,177), dims=['lat_g','lon_g'])
YG = np.real(YG).rename('YG')

CS = xr.DataArray(np.fromfile(LLC_data_folder+"grid/AngleCS_177x201", dtype='>f').reshape(201,177), dims=['lat','lon'])
CS = np.real(CS).rename('CS')

SN = xr.DataArray(np.fromfile(LLC_data_folder+"grid/AngleSN_177x201", dtype='>f').reshape(201,177), dims=['lat','lon'])
SN = np.real(SN).rename('SN')

Depth = xr.DataArray(np.fromfile(LLC_data_folder+"grid/Depth_177x201", dtype='>f').reshape(201,177), dims=['lat','lon'])
Depth = np.real(Depth).rename('Depth')

dxC = xr.DataArray(np.fromfile(LLC_data_folder+"grid/DXC_177x201", dtype='>f').reshape(201,177), dims=['lat','lon_g'])
dxC = np.real(dxC).rename('dxC')

dyC = xr.DataArray(np.fromfile(LLC_data_folder+"grid/DYC_177x201", dtype='>f').reshape(201,177), dims=['lat_g','lon'])
dyC = np.real(dyC).rename('dyC')

dxG = xr.DataArray(np.fromfile(LLC_data_folder+"grid/DXG_177x201", dtype='>f').reshape(201,177), dims=['lat_g','lon'])
dxG = np.real(dxG).rename('dxG')

dyG = xr.DataArray(np.fromfile(LLC_data_folder+"grid/DYG_177x201", dtype='>f').reshape(201,177), dims=['lat','lon_g'])
dyG = np.real(dyG).rename('dyG')

In [7]:
grid = xr.merge([XC,YC,XG,YG,YC,XG,CS,SN,Depth,dxC,dyC,dxG,dyG])
print(grid)

<xarray.Dataset>
Dimensions:  (lat: 201, lat_g: 201, lon: 177, lon_g: 177)
Dimensions without coordinates: lat, lat_g, lon, lon_g
Data variables:
    XC       (lat, lon) >f4 -18.03125 -18.010416 -17.989584 -17.96875 ...
    YC       (lat, lon) float32 47.404716 47.404716 47.404716 47.404716 ...
    XG       (lat_g, lon_g) float32 -18.041666 -18.020834 -18.0 -17.979166 ...
    YG       (lat_g, lon_g) >f4 47.39811 47.39811 47.39811 47.39811 47.39811 ...
    CS       (lat, lon) >f4 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 ...
    SN       (lat, lon) >f4 -2.519282e-13 -0.0 -0.0 2.5199138e-13 ...
    Depth    (lat, lon) >f4 4658.816 4647.708 4628.1807 4609.2144 4592.1665 ...
    dxC      (lat, lon_g) >f4 1567.638 1567.638 1567.638 1567.638 1567.638 ...
    dyC      (lat_g, lon) >f4 1468.746 1468.746 1468.746 1468.746 1468.746 ...
    dxG      (lat_g, lon) >f4 1567.8346 1567.8346 1567.8346 1567.8346 ...
    dyG      (lat, lon_g) >f4 1468.5818 1468.5818 1468.5818 1468.5818 ...


In [8]:
lat = YC.isel(lon=1)
lat = np.real(lat).rename('lat')

lon = XC.isel(lat=1)
lon = np.real(lon).rename('lon')

lat_g = YG.isel(lon_g=1)
lat_g = np.real(lat_g).rename('lat_g')

lon_g = XG.isel(lat_g=1)
lon_g = np.real(lon_g).rename('lon_g')

grid = xr.Dataset(grid,coords={'lon':lon, 'lat':lat, 'lat_g':lat_g, 'lon_g':lon_g})
grid

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xarray/core/dataset.py:374: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  both_data_and_coords = [k for k in data_vars if k in coords]


<xarray.Dataset>
Dimensions:  (lat: 201, lat_g: 201, lon: 177, lon_g: 177)
Coordinates:
  * lon      (lon) >f4 -18.03125 -18.010416 -17.989584 -17.96875 -17.947916 ...
  * lat      (lat) >f4 47.404716 47.417923 47.43113 47.444332 47.45753 ...
  * lat_g    (lat_g) >f4 47.39811 47.41132 47.424526 47.43773 47.45093 ...
  * lon_g    (lon_g) >f4 -18.041666 -18.020834 -18.0 -17.979166 -17.958334 ...
Data variables:
    XC       (lat, lon) >f4 -18.03125 -18.010416 -17.989584 -17.96875 ...
    YC       (lat, lon) float32 47.404716 47.404716 47.404716 47.404716 ...
    XG       (lat_g, lon_g) float32 -18.041666 -18.020834 -18.0 -17.979166 ...
    YG       (lat_g, lon_g) >f4 47.39811 47.39811 47.39811 47.39811 47.39811 ...
    CS       (lat, lon) >f4 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 ...
    SN       (lat, lon) >f4 -2.519282e-13 -0.0 -0.0 2.5199138e-13 ...
    Depth    (lat, lon) >f4 4658.816 4647.708 4628.1807 4609.2144 4592.1665 ...
    dxC      (lat, lon_g) >f4 1567.638 1567.638

In [9]:
grid = grid.assign_coords(XC=grid.XC) 
grid = grid.assign_coords(YC=grid.YC) 
grid = grid.assign_coords(XG=grid.XG) 
grid = grid.assign_coords(YG=grid.YG) 
grid = grid.assign_coords(CS=grid.CS) 
grid = grid.assign_coords(SN=grid.SN) 
grid = grid.assign_coords(Depth=grid.Depth) 
grid = grid.assign_coords(dxC=grid.dxC) 
grid = grid.assign_coords(dyC=grid.dyC) 
grid = grid.assign_coords(dxG=grid.dxG) 
grid = grid.assign_coords(dyG=grid.dyG) 

grid

<xarray.Dataset>
Dimensions:  (lat: 201, lat_g: 201, lon: 177, lon_g: 177)
Coordinates:
    XC       (lat, lon) >f4 -18.03125 -18.010416 -17.989584 -17.96875 ...
    YC       (lat, lon) float32 47.404716 47.404716 47.404716 47.404716 ...
    XG       (lat_g, lon_g) float32 -18.041666 -18.020834 -18.0 -17.979166 ...
    YG       (lat_g, lon_g) >f4 47.39811 47.39811 47.39811 47.39811 47.39811 ...
    CS       (lat, lon) >f4 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 1.0 ...
    SN       (lat, lon) >f4 -2.519282e-13 -0.0 -0.0 2.5199138e-13 ...
    Depth    (lat, lon) >f4 4658.816 4647.708 4628.1807 4609.2144 4592.1665 ...
    dxC      (lat, lon_g) >f4 1567.638 1567.638 1567.638 1567.638 1567.638 ...
    dyC      (lat_g, lon) >f4 1468.746 1468.746 1468.746 1468.746 1468.746 ...
    dxG      (lat_g, lon) >f4 1567.8346 1567.8346 1567.8346 1567.8346 ...
    dyG      (lat, lon_g) >f4 1468.5818 1468.5818 1468.5818 1468.5818 ...
  * lon      (lon) >f4 -18.03125 -18.010416 -17.989584 -17.96875 -

In [11]:
file_out = LLC_data_folder_data_store+'/Grid.nc'
%time grid.to_netcdf(file_out, mode='w')     

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 25.9 ms


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


In [12]:
cluster.scheduler.close()

<Future pending cb=[_make_coroutine_wrapper.<locals>.wrapper.<locals>.<lambda>() at /home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/tornado/gen.py:336]>